In [5]:
# Import librarais Projet SID 21
import dash
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.graph_objs as go
from dash.dependencies import Input, Output
from sklearn import datasets
from sklearn.cluster import KMeans
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd


# 1 Dash Forme

In [39]:

app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

# the style arguments for the sidebar. We use position:fixed and a fixed width
SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "16rem",
    "padding": "2rem 1rem",
    "background-color": "#C0C0C0",
}

# the styles for the main content position it to the right of the sidebar and
# add some padding.
CONTENT_STYLE = {
    "margin-left": "18rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",
}
sidebar = html.Div(
    [
        html.H1("Projet Dashbord M1 SID", className="display-4"),
        html.Hr(),
        html.P(
            "la contribution de I.A dans les maladies ", className="lead"
        ),
        dbc.Nav(
            [
                dbc.NavLink("Cardiovascular disease", href="/", active="exact"),
                dbc.NavLink("Cancer & IA articles", href="/page-1", active="exact"),
                dbc.NavLink("Schizophrenia & IA articles", href="/page-2", active="exact"),
                dbc.NavLink("Alzheimer & IA articles", href="/page-3", active="exact"),
                dbc.NavLink("AIDS & IA articles", href="/page-4", active="exact"),
                dbc.NavLink("Asthma & IA articles", href="/page-5", active="exact"),
            ],
            vertical=True,
            pills=True,
        ),
    ],
    style=SIDEBAR_STYLE,
)

content = html.Div(id="page-content", style=CONTENT_STYLE)

app.layout = html.Div([dcc.Location(id="url"), sidebar, content])


# 2 Data Processing

## 2-1 oracle database & Flask connection 

In [7]:
df=pd.read_csv('/home/sid2020-7/M1 S2/Projet Dashbord/Dashboard_V7.csv')
df.head()

,Unnamed: 0,url_article,Date,Auteur,Titre,Annee,Affiliation,Pays,id_article,Etat,domaine_titre,keywords,Abstract,department,institute,university
0,0,https://pubmed.ncbi.nlm.nih.gov/19164082/#affi...,['2009 Jul;'],"['Po-Whei Huang', 'Cheng-Hsiung Lee']",Automatic classification for pathological pros...,2009,"<div class=""affiliations"">\n<h3 class=""title"">...",taiwan,19164082,NR,aucun,"[['Algorithms'], ['Artificial', 'Intelligence'...",Accurate grading for prostatic c...,['department of computer science and engineeri...,[],[]
1,1,https://pubmed.ncbi.nlm.nih.gov/24614353/#affi...,['2014 Mar 10;'],"['Yi-Lung Ding', 'Yu-Hsuan Shih', 'Fu-Yuan Tsa...",In silico prediction of inhibition of promiscu...,2014,"<div class=""affiliations"">\n<h3 class=""title"">...",taiwan,24614353,NR,aucun,"[['ATP', 'Binding', 'Cassette', 'Transporter',...",Background ...,['department of medical research and teaching'...,['institute of biotechnology'],[]
2,2,https://pubmed.ncbi.nlm.nih.gov/18044554/#affi...,['2007;'],"['Masahiko Nakamoto', 'Naoki Aburaya', 'Kozo K...",Thoracoscopic surgical navigation system for c...,2007,"<div class=""affiliations"">\n<h3 class=""title"">...",japan,18044554,NR,aucun,"[['Algorithms'], ['Artificial', 'Intelligence'...",We have developed a thoracoscopi...,[],[],[]
3,3,https://pubmed.ncbi.nlm.nih.gov/21601186/#affi...,['2011 Dec;'],"['Mahdi Jalili', 'Maria G Knyazeva']",EEG based functional networks in schizophrenia,2011,"<div class=""affiliations"">\n<h3 class=""title"">...",iran,21601186,NR,aucun,"[['Adult'], ['Brain', 'Mapping', 'methods'], [...",Schizophrenia is often considere...,['department of computer engineering'],[],[]
4,4,https://pubmed.ncbi.nlm.nih.gov/21776783/#affi...,['2011 May;'],"['E Ukwatta', 'J Awad', 'D Buchanan', 'G Parra...",Three dimensional ultrasound of carotid athero...,2011,"<div class=""affiliations"">\n<h3 class=""title"">...",canada,21776783,NR,aucun,"[['Algorithms'], ['Artificial', 'Intelligence'...",Purpose Thr...,[],[],[]


In [4]:
# (cardiovascular disease OR cancer OR asthma OR alzheimer
# OR diabetes OR AIDS OR schizophrenia OR Neurodegenerative Diseases)

In [13]:

# df['count_cancer']=df.Abstract.str.count("cancer")
# df['count_schizophrenia']=df.Abstract.str.count("schizophrenia")
# df['count_alzheimer']=df.Abstract.str.count("alzheimer")
# df['count_AIDS']=df.Abstract.str.count("AIDS")
df['count_asthma']=df.Abstract.str.count("asthma")
df

,Unnamed: 0,url_article,Date,Auteur,Titre,Annee,Affiliation,Pays,id_article,Etat,...,Abstract,department,institute,university,count_cancer,count_schizophrenia,count_alzheimer,count_AIDS,count_cardiovascular,count_asthma
0,0,https://pubmed.ncbi.nlm.nih.gov/19164082/#affi...,['2009 Jul;'],"['Po-Whei Huang', 'Cheng-Hsiung Lee']",Automatic classification for pathological pros...,2009,"<div class=""affiliations"">\n<h3 class=""title"">...",taiwan,19164082,NR,...,Accurate grading for prostatic c...,['department of computer science and engineeri...,[],[],0.0,0.0,0.0,0.0,0.0,0.0
1,1,https://pubmed.ncbi.nlm.nih.gov/24614353/#affi...,['2014 Mar 10;'],"['Yi-Lung Ding', 'Yu-Hsuan Shih', 'Fu-Yuan Tsa...",In silico prediction of inhibition of promiscu...,2014,"<div class=""affiliations"">\n<h3 class=""title"">...",taiwan,24614353,NR,...,Background ...,['department of medical research and teaching'...,['institute of biotechnology'],[],1.0,0.0,0.0,0.0,0.0,0.0
2,2,https://pubmed.ncbi.nlm.nih.gov/18044554/#affi...,['2007;'],"['Masahiko Nakamoto', 'Naoki Aburaya', 'Kozo K...",Thoracoscopic surgical navigation system for c...,2007,"<div class=""affiliations"">\n<h3 class=""title"">...",japan,18044554,NR,...,We have developed a thoracoscopi...,[],[],[],2.0,0.0,0.0,0.0,0.0,0.0
3,3,https://pubmed.ncbi.nlm.nih.gov/21601186/#affi...,['2011 Dec;'],"['Mahdi Jalili', 'Maria G Knyazeva']",EEG based functional networks in schizophrenia,2011,"<div class=""affiliations"">\n<h3 class=""title"">...",iran,21601186,NR,...,Schizophrenia is often considere...,['department of computer engineering'],[],[],0.0,1.0,0.0,0.0,0.0,0.0
4,4,https://pubmed.ncbi.nlm.nih.gov/21776783/#affi...,['2011 May;'],"['E Ukwatta', 'J Awad', 'D Buchanan', 'G Parra...",Three dimensional ultrasound of carotid athero...,2011,"<div class=""affiliations"">\n<h3 class=""title"">...",canada,21776783,NR,...,Purpose Thr...,[],[],[],0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9986,9986,https://pubmed.ncbi.nlm.nih.gov/29727276/#affi...,['2018 May;'],"['Zhiwei Wang', 'Danpeng Cheng', 'Xin Yang']",Automated Detection of Clinically Significant ...,2018,NaN,non renseigne,29727276,NR,...,Automated methods for detecting ...,[],[],[],1.0,0.0,0.0,0.0,0.0,0.0
9987,9987,https://pubmed.ncbi.nlm.nih.gov/31527646/#affi...,['2019 Sep 16;'],"['Konobu Kimura', '\n 2\n ...",A novel automated image analysis system using ...,2019,"<div class=""affiliations"">\n<h3 class=""title"">...",japan,31527646,NR,...,Detection of dysmorphic cells in...,['department of next generation hematology lab...,[],[],0.0,0.0,0.0,0.0,0.0,0.0
9988,9988,https://pubmed.ncbi.nlm.nih.gov/29994053/#affi...,['2019 Mar;'],['Jeremy Kawahara'],Fully Convolutional Neural Networks to Detect ...,2019,NaN,non renseigne,29994053,NR,...,The presence of certain clinical...,[],[],[],0.0,0.0,0.0,0.0,0.0,0.0
9989,9989,https://pubmed.ncbi.nlm.nih.gov/20052894/#affi...,['2009 Dec;'],['Rajasvaran Logeswaran'],Cholangiocarcinoma an automated preliminary d...,2009,"<div class=""affiliations"">\n<h3 class=""title"">...",korea,20052894,NR,...,Cholangiocarcinoma cancer of th...,[],[],[],1.0,0.0,0.0,0.0,0.0,0.0


In [14]:
df_dat_them=df[['Annee','count_cardiovascular','count_cancer','count_schizophrenia','count_alzheimer','count_AIDS','count_asthma']]
# df_dat_them

In [20]:
import plotly.express as px
df = px.data.gapminder().query("year == 2007").query("continent == 'Europe'")
df.loc[df['pop'] < 2.e6, 'country'] = 'Other countries' # Represent only large countries
figE = px.pie(df, values='pop', names='country', title='Population of European continent')
figE.show()

ValueError: Trace type 'scatterpolar' is not compatible with subplot type 'xy'
at grid position (1, 2) 

See the docstring for the specs argument to plotly.subplots.make_subplots 
for more information on subplot types

In [33]:
# 'count_cancer'
df2=df_dat_them[['Annee','count_cancer']]
df2= df2[df2['count_cancer'] >1]
df2 = df2.groupby(['Annee']).count_cancer.sum().reset_index()
from plotly.subplots import make_subplots

fig2 = make_subplots(rows=2, cols=3, subplot_titles=['Plot 1','count_cancer', 'count_cancer'])
fig2.add_scatter(x=df2['Annee'], y= df2['count_cancer'], row=1, col=1)
fig2.add_scatter(x=df2['Annee'], y= df2['count_cancer'], row=1, col=3)
fig2.add_bar(x=df2['Annee'], y=df2['count_cancer'], row=1,col=2)
fig2.add_bar(x=df2['Annee'], y=df2['count_cancer'], row=2,col=2)

In [34]:
# 'count_schizophrenia'
df3=df_dat_them[['Annee','count_schizophrenia']]
df3= df3[df3['count_schizophrenia'] >3]
df3 = df3.groupby(['Annee']).count_schizophrenia.sum().reset_index()
from plotly.subplots import make_subplots
fig3 = make_subplots(rows=2, cols=3, subplot_titles=['Plot 1', 'count_schizophrenia', 'Plot 3','count_schizophrenia'])
fig3.add_scatter(x=df3['Annee'], y= df3['count_schizophrenia'], row=1, col=1)
fig3.add_scatter(x=df3['Annee'], y= df3['count_schizophrenia'], row=1, col=3)
fig3.add_bar(x=df3['Annee'], y=df3['count_schizophrenia'], row=1,col=2)
fig3.add_bar(x=df3['Annee'], y=df3['count_schizophrenia'], row=2,col=2)

In [35]:
# 'count_alzheimer'
df4=df_dat_them[['Annee','count_alzheimer']]
df4= df4[df4['count_alzheimer'] >= 1]
fig4 = make_subplots(rows=2, cols=3, subplot_titles=['Plot 1', 'count_alzheimer', 'count_alzheimer','count_alzheimer'])
fig4.add_scatter(x=df4['Annee'], y= df4['count_alzheimer'], row=1, col=1)
fig4.add_scatter(x=df4['Annee'], y= df4['count_alzheimer'], row=1, col=3)
fig4.add_bar(x=df4['Annee'], y=df4['count_alzheimer'], row=1,col=2)
fig4.add_bar(x=df4['Annee'], y=df4['count_alzheimer'], row=2,col=2)

In [36]:
# 'count_AIDS'
df5=df_dat_them[['Annee','count_AIDS']]
df5= df5[df5['count_AIDS'] >= 1]
df5 = df5.groupby(['Annee']).count_AIDS.sum().reset_index()
fig5 = make_subplots(rows=2, cols=3, subplot_titles=['Plot 1', 'count_AIDS', 'count_AIDS','count_alzheimer'])
fig5.add_scatter(x=df5['Annee'], y= df5['count_AIDS'], row=1, col=1)
fig5.add_scatter(x=df5['Annee'], y= df5['count_AIDS'], row=1, col=3)
fig5.add_scatter(x=df5['Annee'], y= df5['count_AIDS'], row=2, col=1)
fig5.add_bar(x=df5['Annee'], y=df5['count_AIDS'], row=1,col=2)
fig5.add_bar(x=df5['Annee'], y=df5['count_AIDS'], row=2,col=2)

In [37]:
# 'count_asthma'
df6=df_dat_them[['Annee','count_asthma']]
df6= df6[df6['count_asthma'] >= 1]
df6 = df6.groupby(['Annee']).count_asthma.sum().reset_index()
fig6 = make_subplots(rows=2, cols=3, subplot_titles=['Plot 1','count_asthma', 'count_asthma', 'fouad','soufiane','amine'])
fig6.add_scatter(x=df6['Annee'], y= df6['count_asthma'], row=1, col=1)
fig6.add_scatter(x=df6['Annee'], y= df6['count_asthma'], row=1, col=3)
fig6.add_scatter(x=df6['Annee'], y= df6['count_asthma'], row=2, col=1)
fig6.add_bar(x=df6['Annee'], y=df6['count_asthma'], row=1,col=2)
fig6.add_bar(x=df6['Annee'], y=df6['count_asthma'], row=2,col=2)
fig6.add_bar(x=df6['Annee'], y=df6['count_asthma'], row=2,col=3)

## 2-2 getting the necessary data to plot the graphes

# 3 Graphes for each disease or IA tech

# 4 pages for each deseas or each IA tech

In [41]:
px1 = html.P([html.Div(children=[
    # All elements from the top of the page
    html.Div([
            html.H1(children='Cardiovascular diseas'),
            dcc.Graph(id='graph1',figure=fig1
            ),  
        ])
   ])
])

###############
px2 = html.P([html.Div(children=[
    # All elements from the top of the page
    html.Div([
            html.H1(children='Cancer'),
            dcc.Graph(id='graph1',figure=fig2
            ),  
        ])
   ])
])
######
px3 = html.P([html.Div(children=[
    # All elements from the top of the page
    html.Div([
            html.H1(children='Schizophrenia'),
            dcc.Graph(id='graph1',figure=fig3
            ),  
        ])
   ])
])

#####
px4 = html.P([html.Div(children=[
    # All elements from the top of the page
    html.Div([
            html.H1(children='Alzheimer'),
            dcc.Graph(id='graph1',figure=fig4
            ),  
        ])
   ])
])

#####
px5 = html.P([html.Div(children=[
    # All elements from the top of the page
    html.Div([
            html.H1(children='AIDS'),
            dcc.Graph(id='graph1',figure=fig5
            ),  
        ])
   ])
])


######
px6 = html.P([html.Div(children=[
    # All elements from the top of the page
    html.Div([
            html.H1(children='Asthma'),
            dcc.Graph(id='graph1',figure=fig6
            ),
        ])
   ])
])

In [42]:
# 5 Run app
@app.callback(Output("page-content", "children"), [Input("url", "pathname")])
def render_page_content(pathname):
    if pathname == "/":
        return px1
    elif pathname == "/page-1":
        return px2
    elif pathname == "/page-2":
        return px3
    elif pathname == "/page-3":
        return px4
    elif pathname == "/page-4":
        return px5
    elif pathname == "/page-5":
        return px6
        
    # If the user tries to reach a different page, return a 404 message
    return dbc.Jumbotron(
        [
            html.H1("404: Not found", className="text-danger"),
            html.Hr(),
            html.P(f"The pathname {pathname} was not recognised..."),
        ]
    )

if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Mar/2021 00:43:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Mar/2021 00:43:59] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [15/Mar/2021 00:43:59] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [15/Mar/2021 00:43:59] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Mar/2021 00:44:03] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Mar/2021 00:44:04] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Mar/2021 00:44:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Mar/2021 00:44:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Mar/2021 00:44:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Mar/2021 00:44:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Mar/2021 00:44:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Mar/2021 00:44:44] "POST /_dash-update-component

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Feb/2021 22:21:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Feb/2021 22:21:44] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [24/Feb/2021 22:21:44] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [24/Feb/2021 22:21:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/Feb/2021 22:21:46] "POST /_dash-update-component HTTP/1.1" 200 -
